Here is an example of how we can load data from github

In [16]:
dataURL = 'https://media.githubusercontent.com/media/jsakhnin/JigsawNLP_data/master/sample_submission.csv'
import pandas as pd
data = pd.read_csv(dataURL)
data.head()

,id,prediction
0,7097320,0.0
1,7097321,0.0
2,7097322,0.0
3,7097323,0.0
4,7097324,0.0
